In [5]:
import os
from dotenv import load_dotenv
import pandas as pd
import mysql.connector
from datetime import datetime

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_port = os.getenv('DB_PORT')
csv_file_path = 'Automated_Traffic_Volume_Counts.csv'  # Asegúrate de actualizar esto con tu archivo CSV correcto

# Verificar si las variables de entorno se cargaron correctamente
if not all([db_host, db_user, db_password, db_name, db_port]):
    raise ValueError("No se pudieron cargar todas las variables de entorno. Por favor verifica el archivo .env.")

# Cargar el archivo CSV en un DataFrame de pandas
try:
    df = pd.read_csv(csv_file_path)
except FileNotFoundError as e:
    raise FileNotFoundError(f"El archivo {csv_file_path} no se encuentra. Por favor verifica la ruta.")

# Filtrar los datos desde enero 2023 hasta agosto 2024
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 8, 31)

# Ensure the columns are in the correct format
df['Yr'] = df['Yr'].astype(int)
df['M'] = df['M'].astype(int)
df['D'] = df['D'].astype(int)
df['HH'] = df['HH'].astype(int)
df['MM'] = df['MM'].astype(int)

# Create a datetime column
df['datetime'] = pd.to_datetime(df[['Yr', 'M', 'D', 'HH', 'MM']].rename(columns={'Yr': 'year', 'M': 'month', 'D': 'day', 'HH': 'hour', 'MM': 'minute'}))

df_filtered = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)].copy()

# Seleccionar solo las columnas necesarias para el análisis
columns_to_keep = ['RequestID', 'Boro', 'Yr', 'M', 'D', 'HH', 'MM', 'Vol', 'SegmentID', 'WktGeom', 'Direction']
df_filtered = df_filtered[columns_to_keep]

# Conectar a la base de datos MySQL
try:
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=db_name,
        port=int(db_port)
    )
except mysql.connector.Error as err:
    raise mysql.connector.Error(f"Error al conectar a la base de datos: {err}")

cursor = db_connection.cursor()

# Insertar los datos filtrados del DataFrame en bloques de 100,000 filas
chunk_size = 100000
rows_inserted = 0
for start in range(0, len(df_filtered), chunk_size):
    end = start + chunk_size
    chunk = df_filtered.iloc[start:end]

    insert_query = """
    INSERT INTO trafico (RequestID, Boro, Yr, M, D, HH, MM, Vol, SegmentID, WktGeom, Direction)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = [(
        row['RequestID'], row['Boro'], row['Yr'], row['M'], row['D'], row['HH'], row['MM'], row['Vol'],
        row['SegmentID'], row['WktGeom'], row['Direction']
    ) for _, row in chunk.iterrows()]

    cursor.executemany(insert_query, data)
    rows_inserted += len(data)
    
    # Imprimir el progreso solo cada 10 bloques
    if start // chunk_size % 10 == 0:
        print(f"Se han insertado {len(data)} filas en este bloque. Total filas insertadas: {rows_inserted}")

# Confirmar los cambios y cerrar la conexión
db_connection.commit()
cursor.close()
db_connection.close()

# Mostrar el número total de filas insertadas
print(f"Se han insertado un total de {rows_inserted} filas en la base de datos.")


Se han insertado 100000 filas en este bloque. Total filas insertadas: 100000
Se han insertado un total de 109248 filas en la base de datos.


## Script de Importación de Temperaturas Promedio de NYC a MySQL


In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import mysql.connector

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_port = os.getenv('DB_PORT')
csv_file_path = 'temperaturas_promedio_nyc_mensual.csv'  # Asegúrate de actualizar esto con tu archivo CSV correcto

# Verificar si las variables de entorno se cargaron correctamente
if not all([db_host, db_user, db_password, db_name, db_port]):
    raise ValueError("No se pudieron cargar todas las variables de entorno. Por favor verifica el archivo .env.")

# Cargar el archivo CSV en un DataFrame de pandas
try:
    df = pd.read_csv(csv_file_path)
except FileNotFoundError as e:
    raise FileNotFoundError(f"El archivo {csv_file_path} no se encuentra. Por favor verifica la ruta.")

# Conectar a la base de datos MySQL
try:
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=db_name,
        port=int(db_port)
    )
except mysql.connector.Error as err:
    raise mysql.connector.Error(f"Error al conectar a la base de datos: {err}")

cursor = db_connection.cursor()

# Insertar los datos del DataFrame en la tabla MySQL
insert_query = """
INSERT INTO temperaturas (Mes, Manhattan, Brooklyn, Queens, The_Bronx, Staten_Island)
VALUES (%s, %s, %s, %s, %s, %s)
"""
data = [tuple(row) for row in df.values]

cursor.executemany(insert_query, data)

# Confirmar los cambios y cerrar la conexión
db_connection.commit()
cursor.close()
db_connection.close()

# Mostrar el número de filas insertadas
print(f"Se han insertado {len(data)} filas en la base de datos.")
